In [81]:
import lmdb
import numpy as np
import cv2
from caffe.proto.caffe_pb2 import Datum
from tqdm import tqdm
import os

In [82]:
env = lmdb.open('./mnist_train_lmdb/', map_size=80000*28*28*5)

In [83]:
img_dir = '../../buff_benchmark/red_number_new/'

In [84]:
with env.begin() as txn:
    print txn.stat()

{'branch_pages': 68L, 'leaf_pages': 15000L, 'overflow_pages': 0L, 'psize': 4096L, 'depth': 3L, 'entries': 60000L}


In [85]:
with env.begin(write=True) as txn:
    i = 60000
    for img_name in tqdm(os.listdir(img_dir)):
        if not img_name.endswith('.jpg'):
            continue
        img_path = os.path.join(img_dir, img_name)
        img = cv2.imread(img_path)
        datum = Datum()
        datum.data = img.tobytes()
        datum.width = 28
        datum.height = 28
        datum.channels = 1
        datum.label = int(img_name.split('_')[1].split('.')[0])
        txn.put('{:08}'.format(i), datum.SerializeToString())
        i += 1

100%|██████████| 2210/2210 [00:01<00:00, 1818.77it/s]


In [86]:
with env.begin() as txn:
    print txn.stat()

{'branch_pages': 68L, 'leaf_pages': 15014L, 'overflow_pages': 2048L, 'psize': 4096L, 'depth': 3L, 'entries': 62048L}
